TEST

In [1]:
import sys
sys.path.append('..')
%load_ext autoreload
%autoreload 2
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [1]:
n_images = 10000
n_test = int(n_images/5)
n_train = int(n_images)*2

In [2]:
import numpy as np
import os

image_width = 45
image_height = 45

images = np.empty(shape=[0,image_width,image_height])
images2 = np.empty(shape=[0,image_width,image_height])

from astropy.io import fits
from astropy.utils.data import get_pkg_data_filename

filename = os.listdir(os.getcwd()+"/data_of_lsst/lsst_mocks_single/lensed_outputs/0")
filename2 = os.listdir(os.getcwd()+"/data_of_lsst/lsst_mocks_single/unlensed_outputs/0")

for file in filename[:n_images]:
    if not file.startswith('.'):
        try:
            image_file = get_pkg_data_filename("data_of_lsst/lsst_mocks_single/lensed_outputs/0/" + file)
            image_data = fits.getdata(image_file, ext=0, ignore_missing_end=True, padding=False)
            images = np.concatenate((images, [image_data]))
        except OSError as err:
            print('This file sucks %s' % (image_file))

            
print ("Done !")

for file in filename2[:n_images]:
    if not file.startswith('.'):
        try:
            image_file = get_pkg_data_filename("data_of_lsst/lsst_mocks_single/unlensed_outputs/0/" + file)
            image_data = fits.getdata(image_file, ext=0, ignore_missing_end=True, padding=False)
            images2 = np.concatenate((images2, [image_data]))
        except OSError as err:
            print('This file sucks %s' % (image_file))

print ("Done !")

lensed_output_0 = images
unlensed_output_0 = images2

Done !
Done !


Creating training and validation sets

In [4]:
# Remove 'nan' from datasets
lensed_output_0 = np.where(np.isfinite(lensed_output_0), lensed_output_0, 0)
unlensed_output_0 = np.where(np.isfinite(unlensed_output_0), unlensed_output_0, 0)

In [10]:
import random 
from keras.utils import np_utils

# set aside data for testing
training_data= np.concatenate((lensed_output_0, unlensed_output_0), axis=0)
# testing labels
training_labels = np.concatenate((np.ones(n_images), np.zeros(n_images)), axis=0)

combined = list(zip(training_data,training_labels))
random.shuffle(combined)
training_data, training_labels = zip(*combined)
training_data = np.array(training_data)
training_labels = np.array(training_labels)

X_train = training_data[:int(n_images)*2]
y_train = training_labels[:int(n_images)*2]
X_test = training_data[16000:n_train]
y_test = training_labels[16000:n_train]


# Classic CNN

In [11]:
import tensorflow as tf
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv1D, MaxPooling1D, GlobalAveragePooling1D
from keras.utils import np_utils
from keras.optimizers import SGD, RMSprop
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Conv2D, MaxPooling2D

In [12]:
nb_classes = 2

# input data dimensions
data_len = 45

# reshape X data
img_cols = 45
img_rows = 45
input_shape=(img_cols, img_rows, 1)

X_train = X_train.reshape(X_train.shape[0], img_cols, img_rows, 1)
X_test = X_test.reshape(X_test.shape[0], img_cols, img_rows, 1)

# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

In [13]:
# Sequential Model

model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(4, 4)))
model.add(Flatten())
model.add(Dropout(0.1))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

model.summary()
model.compile(loss='binary_crossentropy', optimizer=RMSprop(), metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 43, 43, 32)        320       
_________________________________________________________________
activation_3 (Activation)    (None, 43, 43, 32)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 10, 10, 32)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 3200)              0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 3200)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 6402      
_________________________________________________________________
activation_4 (Activation)    (None, 2)                 0         
Total para

In [17]:
import matplotlib.pyplot as plt

### train and evaluate CNN
print('')
batch_size=97
n_epochs=5
model.fit(X_train, Y_train, batch_size=batch_size, epochs=n_epochs, 
          validation_data=(X_test, Y_test))
score = model.evaluate(X_test, Y_test, batch_size=batch_size, verbose=0)
print('')
print('This CNN has an accuracy of about' + '% .0f' % (score[1]*100) + '%.')
print('Typical accuracies for this training set fall around 50%.')


Train on 19999 samples, validate on 3999 samples
Epoch 1/5
19999/19999 [==============================] - 18s 919us/step - loss: 8.0147 - acc: 0.5000 - val_loss: 7.8848 - val_acc: 0.5081
Epoch 2/5
19999/19999 [==============================] - 18s 887us/step - loss: 8.0147 - acc: 0.5000 - val_loss: 7.8848 - val_acc: 0.5081
Epoch 3/5
19999/19999 [==============================] - 18s 876us/step - loss: 8.0147 - acc: 0.5000 - val_loss: 7.8848 - val_acc: 0.5081
Epoch 4/5
19999/19999 [==============================] - 18s 884us/step - loss: 8.0147 - acc: 0.5000 - val_loss: 7.8848 - val_acc: 0.5081
Epoch 5/5
19999/19999 [==============================] - 18s 895us/step - loss: 8.0147 - acc: 0.5000 - val_loss: 7.8848 - val_acc: 0.5081

This CNN has an accuracy of about 51%.
Typical accuracies for this training set fall around 50%.


In [75]:
list(zip(model.predict(X_test[:20]).T[1], model.predict_classes(X_test[:20]), y_test[0:20]))

[(0.52784616, 1, 1.0),
 (0.5249349, 1, 1.0),
 (0.54977274, 1, 1.0),
 (0.5042342, 1, 1.0),
 (0.51869714, 1, 0.0),
 (0.5266753, 1, 0.0),
 (0.5320569, 1, 0.0),
 (0.5346558, 1, 0.0),
 (0.51286584, 1, 1.0),
 (0.5661322, 1, 0.0),
 (0.5243514, 1, 1.0),
 (0.54082507, 1, 1.0),
 (0.5099284, 1, 0.0),
 (0.53747445, 1, 1.0),
 (0.54854506, 1, 1.0),
 (0.52364546, 1, 1.0),
 (0.5478093, 1, 1.0),
 (0.5587346, 1, 0.0),
 (0.5510385, 1, 0.0),
 (0.50517875, 1, 1.0)]